In [1]:
import uproot#3 as uproot
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as pl

import k3d

In [2]:
from modules.ML4Pions_Dataset import MLPionsDataset_FixedR, MLPionsDataset_KNN

Using backend: pytorch


In [3]:
!ls -a 

.
..
.DS_Store
.ipynb_checkpoints
ML4Pions_Data.ipynb
ML4Pions_Data.py
ML4Pions_Dataset_Old.ipynb
ML4Pions_Dataset_display.ipynb
SLIDES
__pycache__
fixed_radius_graph.py
ml4pions_event_display-master
ml4pions_event_display-master.zip
modules
root.log
user.angerami.24559744.OutputStream._000335.root


In [4]:
dataset = MLPionsDataset_KNN(filename='user.angerami.24409108.OutputStream._000257.root', k_val=5)

100%|██████████| 20000/20000 [03:37<00:00, 91.75it/s] 


In [5]:
dataset_fr = MLPionsDataset_FixedR(filename='user.angerami.24409108.OutputStream._000257.root', radius=1000, n_neighbor=20)

100%|██████████| 20000/20000 [04:25<00:00, 75.34it/s]


In [7]:
ev_idx = 5

dataset[ev_idx]['gr'] # -- list of KNN graph, one for each cluster

[Graph(num_nodes=212, num_edges=1060,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=156, num_edges=780,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=15, num_edges=75,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=6, num_edges=30,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=4, num_edges=16,
       ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={})]

In [8]:
dataset_fr[ev_idx]['gr'] # -- list of FR graph, one for each cluster

[Graph(num_nodes=212, num_edges=4240,
       ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'center': Scheme(shape=(), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=156, num_edges=3120,
       ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'center': Scheme(shape=(), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=15, num_edges=225,
       ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'center': Scheme(shape=(), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32), 'en': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={}),
 Graph(num_nodes=6, num_edges=36,
       ndata_schemes={'pos': Scheme(shape=(3,), dtype=torch.float32), 'center': Scheme(shape=(), dtype=torch.float32), 'x': Scheme(shape=(3,), dty

In [9]:
gr_knn, gr_fr = dataset[ev_idx]['gr'][2], dataset_fr[ev_idx]['gr'][2]

In [35]:
def plot_graph(gr, name='') : 
    
    cell_E = gr.ndata['en'].numpy()
    min_e,max_e = np.min(cell_E), np.max(cell_E)
    
    cluster_cell_pos = gr.ndata['x'].numpy()
    
    colormap = k3d.colormaps.basic_color_maps.CoolWarm
    cell_colors = k3d.helpers.map_colors(np.log(cell_E),colormap,[min_e,max_e])
    cell_colors = cell_colors.astype(np.int32)
    
    plot = k3d.plot(name=name)
    
    cell_p = k3d.points(positions=cluster_cell_pos,point_size=10,
                        colors=cell_colors.astype(np.int32))
    
    cell_p.shader='3d'
    plot += cell_p
    
    U, V = gr.edges()
    
    for edge_idx in range( len(U) ) : 
        e_st = cluster_cell_pos[ U[edge_idx].item() ]
        e_en = cluster_cell_pos[ V[edge_idx].item() ]
        plt_line = k3d.line([e_st,e_en],shader='mesh', width=1., color=0xff0000)
        plot += plt_line
    
    plot.display()
    
#     print(cluster_cell_pos.shape)
#     print(type(cluster_cell_pos))

In [36]:
plot_graph(gr = gr_fr, name='fixedR')

Output()

In [37]:
plot_graph(gr = gr_knn, name='KNN')

Output()